In [ ]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.utils as vutils



In [ ]:
transforms=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

In [ ]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transforms,download=True)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transforms,download=True)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)


100%|██████████| 170M/170M [00:57<00:00, 2.96MB/s]


In [ ]:
image,label=train_data[0]

In [ ]:
image.size()


torch.Size([3, 32, 32])

In [ ]:
class_names=["plane","car","bird","cat","deer","dog","frog","horse","ship","truck"]

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        x = self.fc3(x)
        return x





In [ ]:
net=NeuralNet()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [ ]:
for epoch in range(30):
    print(f"Training epoch {epoch+1}")

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Loss: {epoch_loss:.4f}")


Training epoch 1
Loss: 2.1379
Training epoch 2
Loss: 1.7919
Training epoch 3
Loss: 1.5773
Training epoch 4
Loss: 1.4541
Training epoch 5
Loss: 1.3740
Training epoch 6
Loss: 1.3004
Training epoch 7
Loss: 1.2347
Training epoch 8
Loss: 1.1757
Training epoch 9
Loss: 1.1158
Training epoch 10
Loss: 1.0634
Training epoch 11
Loss: 1.0141
Training epoch 12
Loss: 0.9740
Training epoch 13
Loss: 0.9325
Training epoch 14
Loss: 0.8988
Training epoch 15
Loss: 0.8628
Training epoch 16
Loss: 0.8315
Training epoch 17
Loss: 0.8000
Training epoch 18
Loss: 0.7671
Training epoch 19
Loss: 0.7400
Training epoch 20
Loss: 0.7088
Training epoch 21
Loss: 0.6809
Training epoch 22
Loss: 0.6512
Training epoch 23
Loss: 0.6277
Training epoch 24
Loss: 0.6034
Training epoch 25
Loss: 0.5790
Training epoch 26
Loss: 0.5519
Training epoch 27
Loss: 0.5303
Training epoch 28
Loss: 0.5104
Training epoch 29
Loss: 0.4874
Training epoch 30
Loss: 0.4640


In [ ]:
torch.save(net.state_dict(),'trained_net.pth')


In [ ]:
net=NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [ ]:
correct=0
total=0
net.eval()
with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=net(images)
    _, predicted = torch.max(outputs, 1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
accuracy=100*correct/total
print(f'Accuracy: {accuracy}%')

Accuracy: 75.18%


In [ ]:
from torch.utils.data import dataloader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data

def load_data(data_augmentation=False):
  transforms_augmented=transforms.Compose([
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(10),
      transforms.ToTensor(),
      transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
  ])

  original_transforms = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
  ])

  if data_augmentation:

    original_datasets = datasets.CIFAR10(root='./data',train=True,transform=original_transforms,download=True)
    augmented_dataset = datasets.CIFAR10(root='./data',train=True,download=True,transform=transforms_augmented)
    train_dataset = torch.utils.data.ConcatDataset([original_datasets,augmented_dataset])
  else:
    train_dataset = datasets.CIFAR10(root='./data',train=True,transform=original_transforms,download=True)

  test_dataset = datasets.CIFAR10(root='./data',train=False,transform=original_transforms,download=True)
  train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True,num_workers=2)
  test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=False,num_workers=2)
  return train_loader,test_loader

In [ ]:
from typing_extensions import evaluate_forward_ref
epochs=25
train_loader,test_loader=load_data(data_augmentation=True)
model=NeuralNet()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)


def train_model(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

print("With augmentation")
for epoch in range(epochs):
  loss=train_model(model,train_loader,criterion,optimizer)
  print(f"Epoch: {epoch+1}, Loss: {loss:.4f}")
accuracy=evaluate_model(model,test_loader)
print(f"Accuracy: {accuracy:.2f}%")

With augmentation
Epoch: 1, Loss: 1.4527
Epoch: 2, Loss: 1.1028
Epoch: 3, Loss: 0.9706
Epoch: 4, Loss: 0.8888
Epoch: 5, Loss: 0.8356
Epoch: 6, Loss: 0.7943
Epoch: 7, Loss: 0.7615
Epoch: 8, Loss: 0.7320
Epoch: 9, Loss: 0.7033
Epoch: 10, Loss: 0.6875
Epoch: 11, Loss: 0.6710
Epoch: 12, Loss: 0.6573
Epoch: 13, Loss: 0.6416
Epoch: 14, Loss: 0.6289
Epoch: 15, Loss: 0.6193
Epoch: 16, Loss: 0.6099
Epoch: 17, Loss: 0.5992
Epoch: 18, Loss: 0.5867
Epoch: 19, Loss: 0.5817
Epoch: 20, Loss: 0.5716
Epoch: 21, Loss: 0.5625
Epoch: 22, Loss: 0.5568
Epoch: 23, Loss: 0.5508
Epoch: 24, Loss: 0.5422
Epoch: 25, Loss: 0.5364
Accuracy: 77.44%
